In [11]:
# 🧱 Cell 0 — Install + Imports (MobileNetV2 Pipeline)

!pip install -q tensorflow opencv-python pillow tqdm kagglehub

import os, shutil, random
from pathlib import Path

import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import kagglehub

print("✅ Setup complete!")
print("TensorFlow version:", tf.__version__)


✅ Setup complete!
TensorFlow version: 2.19.0


In [12]:
# 🧱 Cell 1 — Download dataset + set SRC_DIR

print("📥 Downloading dataset...")
path = kagglehub.dataset_download("gti-upm/leapgestrecog")
print("Dataset path:", path)

DATASET_ROOT = Path(path)
SRC_DIR = DATASET_ROOT / "leapGestRecog" if (DATASET_ROOT / "leapGestRecog").exists() else DATASET_ROOT

subjects = sorted([d.name for d in SRC_DIR.iterdir() if d.is_dir()])

print("✅ Using dataset from:", SRC_DIR)
print("✅ Subjects found:", subjects)
print("✅ Number of subjects:", len(subjects))


📥 Downloading dataset...
Using Colab cache for faster access to the 'leapgestrecog' dataset.
Dataset path: /kaggle/input/leapgestrecog
✅ Using dataset from: /kaggle/input/leapgestrecog/leapGestRecog
✅ Subjects found: ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']
✅ Number of subjects: 10


In [13]:
# 🧱 Cell 2 — Subject-based split (80/10/10) + Flatten by Gesture

random.seed(42)

OUT_DIR = Path("gesture_data")
TRAIN_DIR = OUT_DIR / "train"
VAL_DIR   = OUT_DIR / "val"
TEST_DIR  = OUT_DIR / "test"

# reset output folder
if OUT_DIR.exists():
    print("ℹ️ Removing old gesture_data...")
    shutil.rmtree(OUT_DIR)

TRAIN_DIR.mkdir(parents=True, exist_ok=True)
VAL_DIR.mkdir(parents=True, exist_ok=True)
TEST_DIR.mkdir(parents=True, exist_ok=True)

subjects = sorted([d.name for d in SRC_DIR.iterdir() if d.is_dir()])
random.shuffle(subjects)

n = len(subjects)
n_train = int(n * 0.8)
n_val   = int(n * 0.1)

train_sub = subjects[:n_train]
val_sub   = subjects[n_train:n_train + n_val]
test_sub  = subjects[n_train + n_val:]

print("📌 Split:")
print("Train subjects:", train_sub)
print("Val subjects  :", val_sub)
print("Test subjects :", test_sub)

def copy_subjects_to_split(sub_list, target_root: Path):
    for s in sub_list:
        subj_path = SRC_DIR / s
        for gesture_dir in subj_path.iterdir():
            if not gesture_dir.is_dir():
                continue
            gesture_name = gesture_dir.name  # e.g., 01_palm
            (target_root / gesture_name).mkdir(parents=True, exist_ok=True)

            for img in gesture_dir.iterdir():
                if img.is_file():
                    dst = target_root / gesture_name / f"{s}_{img.name}"
                    shutil.copy2(img, dst)

print("\n🚀 Copying Train...")
copy_subjects_to_split(train_sub, TRAIN_DIR)

print("🚀 Copying Val...")
copy_subjects_to_split(val_sub, VAL_DIR)

print("🚀 Copying Test...")
copy_subjects_to_split(test_sub, TEST_DIR)

def count_images(root: Path):
    total = 0
    for cls in root.iterdir():
        if cls.is_dir():
            total += len([f for f in cls.iterdir() if f.is_file()])
    return total

print("\n✅ gesture_data ready!")
print("Train images:", count_images(TRAIN_DIR))
print("Val images  :", count_images(VAL_DIR))
print("Test images :", count_images(TEST_DIR))

print("✅ Num gesture classes (train):", len([d for d in TRAIN_DIR.iterdir() if d.is_dir()]))


ℹ️ Removing old gesture_data...
📌 Split:
Train subjects: ['07', '03', '02', '08', '05', '06', '09', '04']
Val subjects  : ['00']
Test subjects : ['01']

🚀 Copying Train...
🚀 Copying Val...
🚀 Copying Test...

✅ gesture_data ready!
Train images: 16000
Val images  : 2000
Test images : 2000
✅ Num gesture classes (train): 10


In [19]:
# 🧱 Cell 3 — Stronger Augmentation (to avoid high val in epoch 1)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

DATA_DIR = "gesture_data"
train_path = os.path.join(DATA_DIR, "train")
val_path   = os.path.join(DATA_DIR, "val")
test_path  = os.path.join(DATA_DIR, "test")

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.20,
    height_shift_range=0.20,
    zoom_range=0.25,
    shear_range=0.15,
    horizontal_flip=True,
    brightness_range=(0.70, 1.30),
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

val_gen = val_test_datagen.flow_from_directory(
    val_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_gen = val_test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("✅ Generators ready!")
print("Num classes:", train_gen.num_classes)


Found 16000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
✅ Generators ready!
Num classes: 10


In [20]:
# 🧱 Cell 4 — MobileNetV2 (weaker head + more dropout)

num_classes = train_gen.num_classes

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)

# Head أضعف + Dropout أعلى
x = Dense(64, activation="relu", kernel_regularizer=l2(5e-4))(x)
x = Dropout(0.70)(x)

outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,340,618 (8.93 MB)

 Trainable params: 82,634 (322.79 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [21]:
# 🧱 Cell 5 — Compile + Callbacks (NO EarlyStopping)

loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.20)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=loss_fn,
    metrics=["accuracy"]
)

CHECKPOINT_PATH = "MobileNetV2_best.keras"  # ✅ Keras format أفضل من .h5

checkpoint_cb = ModelCheckpoint(
    CHECKPOINT_PATH,
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-7
)

callbacks = [checkpoint_cb, reduce_lr]

print("✅ Compiled + callbacks ready!")
print("Checkpoint:", CHECKPOINT_PATH)


✅ Compiled + callbacks ready!
Checkpoint: MobileNetV2_best.keras


In [22]:
# 🧱 Cell 6 — Training (NO EarlyStopping)

EPOCHS = 30

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step - accuracy: 0.2290 - loss: 2.3651
Epoch 1: val_accuracy improved from -inf to 0.79300, saving model to MobileNetV2_best.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 252s 486ms/step - accuracy: 0.2293 - loss: 2.3646 - val_accuracy: 0.7930 - val_loss: 1.5817 - learning_rate: 1.0000e-04
Epoch 2/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.5742 - loss: 1.7246
Epoch 2: val_accuracy improved from 0.79300 to 0.88100, saving model to MobileNetV2_best.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 239s 478ms/step - accuracy: 0.5743 - loss: 1.7245 - val_accuracy: 0.8810 - val_loss: 1.3747 - learning_rate: 1.0000e-04
Epoch 3/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step - accuracy: 0.7005 - loss: 1.5706
Epoch 3: val_accuracy did not improve from 0.88100
500/500 ━━━━━━━━━━━━━━━━━━━━ 235s 471ms/step - accuracy: 0.7005 - loss: 1.5705 - val_accuracy: 0.8740 - val_loss: 1.3102 - learning_rate: 1.0000e-04
Epoch 4/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/st

KeyboardInterrupt: 